In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
exec(open("plot_utils.py").read())

/Users/adsurbum/anaconda2/envs/py371/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
country_name = 'Sweden'
file_name = '../output/2020-05-12-fixed-deltat0-new-normal-100k/inference/{}.npz'.format(country_name)
data = np.load(file_name)



In [4]:
var_names = list(data['var_names'])
incidences = data['incidences']
chain = data['chain']
nsteps, ndim, N, Td1, Td2, τ_model = data['params']
nburn = int(nsteps * 0.6)
sample = chain[:, nburn:, :].reshape(-1, ndim)

medians = [np.median(sample[:,i]) for i in range(len(var_names))]
means = [sample[:,i].mean() for i in range(len(var_names))]
lnprobability = data['lnprobability'][:, nburn:]
MAP = sample[lnprobability.argmax()]

In [5]:
list(zip(var_names, means, medians))

[('Z', 3.854167140482578, 3.994927018906112),
 ('D', 3.4201515516543233, 3.411179005448763),
 ('μ', 0.4483497287149728, 0.3881799803223308),
 ('β', 1.0835115491413074, 1.050978495216374),
 ('α1', 0.16308341875155768, 0.1135492559481953),
 ('λ', 0.6791440211115766, 0.6843530936173623),
 ('α2', 0.29477082240443336, 0.23423300687219878),
 ('E0', 681.0155238953887, 448.98200984421044),
 ('Iu0', 823.952964290953, 578.0630227657103),
 ('Δt0', 2.4761021305700637, 2.042487360261745),
 ('τ', 18.09678803110381, 17.83867986682796)]

## medians

In [73]:
seed_max = 3000
params_bounds = {
    'Z' : (2, 5),
    'D' : (2, 5),
    'μ' : (0.2, 1),
    'β' : (0.8, 1.5),
    'α1' : (0.02, 1),
    'λ'  : (0, 1),
    'α2' : (0.02, 1),
    'E0' : (0, seed_max),
    'Iu0' : (0, seed_max),
    'Δt0' : (1,5) #how much zeros before the first incident
}

## loglikelihood code:
θ = medians
X = incidences
Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays, N)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(0, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])

In [70]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr]
def r_arrays(arr):
    return [r(a) for a in arr]

print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n']))
arr = r_arrays(list(zip(range(len(X)),loglik,X, Xsum, Y, n)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n
0	-2	0	0	0	14
1	-1	0	0	14	31
2	-2	1	1	31	49
3	-7	5	6	50	65
4	-9	5	11	71	83
5	-12	1	12	94	106
6	-4	1	13	118	132
7	-4	1	14	145	160
8	-2	9	23	174	183
9	-38	11	34	206	205
10	-2	26	60	239	216
11	-9	76	136	276	179
12	-8	24	160	315	197
13	-24	42	202	357	155
14	-85	45	247	357	200
15	-146	78	325	447	212
16	-204	136	461	537	168
17	-301	158	619	629	104
18	-inf	155	774	723	46
19	-inf	149	923	820	0
20	-96	108	1031	921	0
21	-191	89	1120	1025	14
22	-190	46	1166	1134	80
23	-404	134	1300	1246	63
24	-inf	122	1422	1363	62
25	-inf	200	1622	1484	0
26	-inf	123	1745	1610	0
27	-1283	160	1905	1741	0
28	-inf	110	2015	1877	3
29	-inf	256	2271	2018	0
30	-inf	238	2509	2165	0


## means

In [10]:
## loglikelihood code:
θ = means
X = incidences
Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays, N)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(0, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])

In [11]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr]
def r_arrays(arr):
    return [r(a) for a in arr]

print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n']))
arr = r_arrays(list(zip(range(len(X)),loglik,X, Xsum, Y, n)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n
0	-6	0	0	0	33
1	-5	0	0	33	73
2	-8	1	1	73	120
3	-21	5	6	121	168
4	-28	5	11	174	224
5	-37	1	12	235	292
6	-27	1	13	304	368
7	-32	1	14	381	453
8	-21	9	23	467	540
9	-3	11	34	563	637
10	-35	26	60	671	732
11	-25	76	136	792	791
12	-31	24	160	927	919
13	-32	42	202	1079	1047
14	-17	45	247	1249	1002
15	-19	78	325	1249	1250
16	-33	136	461	1575	1448
17	-54	158	619	1909	1640
18	-118	155	774	2259	1854
19	-181	149	923	2628	2098
20	-311	108	1031	3021	2408
21	-217	89	1120	3439	2765
22	-282	46	1166	3885	3195
23	-228	134	1300	4361	3568
24	-389	122	1422	4868	3988
25	-394	200	1622	5410	4367
26	-556	123	1745	5989	4862
27	-393	160	1905	6607	5361
28	-481	110	2015	7266	5955
29	-470	256	2271	7970	6451
30	-627	238	2509	8722	7015
